# Seq2Seq DQ Test Notebook

In this notebook we test the dq client for Seq2Seq using simulated / fake data. The main intention is to battle test the different components of the client without training an actual model - i.e. optimizing for speed!

Things that we want to test:
1. Setting the tokenizer
2. Logging data (input + target outputs)
3. Logging model outputs 1+ epoch
4. Fake model generations - interestingly the best way to do this may be with a small validation dataset + a real LLM model. This depends a bit on design decisions around logging for generation.

NOTE: For a first pass we work with just a training dataset

Let's get testing

In [ ]:
# from transformers import T5Tokenizer, T5ForConditionalGeneration
from datasets import load_dataset, Dataset
import numpy as np
# import torch

# TODO: Add dq import from local

## Pull data from hf hub

Since part of the dq processing involves tokenizing and aligning text / token indices, we work with a small real-world dataset - rather than dummy data.

The Billsum dataset contains three columns:

<p style="text-align: center;">|| text || summary || title ||</p>

We look at just **summary** and **title** and map them as follows:
<p style="text-align: center;">(summary, title) --> (input context,  target output)</p>

We also use a small subset of the first 100(0?) data rows!

In [ ]:
dataset_size = 100

ds = load_dataset("billsum")
ds = ds.remove_columns('text')
# Add ids
ds = ds.map(lambda _, idx: {"id": idx}, with_indices=True)
ds_train = Dataset.from_dict(ds['train'][:100])
ds_val = Dataset.from_dict(ds['test'][:100])
ds_train

In [ ]:
ds_train[0]

## Logging Data

1. Before logging input data log the tokenizer (making sure we use the fast tokenizer)
2. Log the input and target output data

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("t5-small", use_fast=True)

# Tokenize things
def tokenize_outputs(row):
    label_ids = tokenizer(row['title'])['input_ids']
    return {'labels': label_ids}

ds_train = ds_train.map(tokenize_outputs)
ds_val = ds_val.map(tokenize_outputs)

In [ ]:
ds_train[0]

In [ ]:
import os
# os.environ['GALILEO_CONSOLE_URL']="http://localhost:8088"
# os.environ["GALILEO_USERNAME"]="user@example.com"
# os.environ["GALILEO_PASSWORD"]="Th3secret_"

import dataquality as dq
dq.configure()
dq.init("seq2seq")
dq.set_tokenizer(tokenizer)

In [ ]:
def log_dataset(ds, input_col="summary", target_output_col="title"):
    dq.log_dataset(
        ds,
        text=input_col,
        label=target_output_col,
        split="training"
    )

# Log just for training
log_dataset(ds_train)

## Logging Model Outputs
Log 1 epoch of fake model output data: includes just logits!

In [ ]:
num_logits = len(tokenizer)


def log_epoch(ds):
    ids = ds['id']
    max_seq_length = np.max([len(ids) for ids in ds['labels']])
    print("len ids", len(ids))
    print("max seq len", max_seq_length)
    # Shape - [bs, max_seq_len, num_logits]
    fake_logits = np.random.randn(len(ids), max_seq_length, num_logits)
    dq.log_model_outputs(
        logits = fake_logits,
        ids = ids
    )

dq.set_epoch(0)
dq.set_split("train")
log_epoch(ds_train)

In [ ]:
dq.finish()